In [1]:
%run ../../notebooks/__init__.ipynb

/cluster/project/beltrao/jjaenes/software/miniconda3/envs/af2genomics-env/lib/python3.9/site-packages/Bio/pairwise2.py:278: BiopythonDeprecationWarning: Bio.pairwise2 has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.PairwiseAligner as a replacement, and contact the Biopython developers if you still need the Bio.pairwise2 module.
  warnings.warn(


In [2]:
l_pssm = ['P00533-F1', 'P04626-F1', 'P04637-F1', 'Q15648-F1', 'O00244-F1', 'Q9Y5Z9-F1', ]
df_pssm = pd.concat(map(lambda struct_id: pd.read_csv(pfile(struct_id=struct_id, step='af2.trim_bf.repairpdb.pssm', suffix='.tsv', base=''), sep='\t'), l_pssm), axis=0)
cols_empty = []
for col in df_pssm.columns:
    if df_pssm[col].nunique() == 1:
        print(col)
        cols_empty.append(col)

df_pssm = df_pssm.drop(['aa_ref', 'aa_pos', 'aa_alt', 'pssm_pos', 'Pdb'] + cols_empty, axis=1)
df_pssm['variant_id'] = df_pssm['variant_id'].str.replace('-F1', '')
df_pssm

chain
SD
sloop_entropy
mloop_entropy
water bridge
electrostatic kon
partial covalent bonds
Entropy Complex


,variant_id,total energy,Backbone Hbond,Sidechain Hbond,Van der Waals,Electrostatics,Solvation Polar,Solvation Hydrophobic,Van der Waals clashes,entropy sidechain,entropy mainchain,cis_bond,torsional clash,backbone clash,helix dipole,disulfide,energy Ionisation
0,P00533/K29A,0.259316,1.039020,0.863101,0.645481,-0.015303,-1.184350,0.788022,-0.007773,-1.375760,-0.484172,0.0,-0.021875,-0.018474,0.012927,0.0,0.0
1,P00533/K29C,0.619045,0.961476,0.863101,0.223816,0.154480,-0.633215,0.367346,-0.007773,-0.968393,-0.334406,0.0,-0.020315,0.025820,0.012927,0.0,0.0
2,P00533/K29D,0.402834,0.366827,0.341720,0.443973,0.060787,-0.605891,0.627430,-0.007899,-0.852986,0.004916,0.0,-0.009186,0.012676,0.033144,0.0,0.0
3,P00533/K29E,0.363608,0.138926,0.109867,0.107757,-0.010313,-0.122570,0.186013,-0.006306,-0.056120,0.007407,0.0,-0.023989,0.003626,0.032935,0.0,0.0
4,P00533/K29F,0.550734,0.845414,0.863101,-0.221429,-0.014297,0.111640,-0.241121,0.004470,-0.666604,-0.126029,0.0,-0.017340,0.061666,0.012927,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5995,Q9Y5Z9/I338S,0.905005,0.018868,0.000000,0.814935,0.000000,-0.735512,1.584240,-0.004999,-0.511685,-0.256442,0.0,-0.004398,-0.219400,0.000000,0.0,0.0
5996,Q9Y5Z9/I338T,0.786325,0.000000,0.000000,0.586331,0.000000,-0.417606,1.158290,0.017524,-0.423065,-0.135071,0.0,-0.000077,-0.141957,0.000000,0.0,0.0
5997,Q9Y5Z9/I338V,0.620887,0.000000,0.000000,0.330322,0.000000,-0.114172,0.735855,0.015844,-0.290465,-0.056418,0.0,-0.000081,-0.099696,0.000000,0.0,0.0
5998,Q9Y5Z9/I338W,-1.683120,0.018868,0.000000,-0.680052,-0.011924,-0.173973,-1.221620,0.025124,0.571906,-0.210463,0.0,-0.000980,-0.008862,0.000000,0.0,0.0


In [3]:
frame_clinvar = merge_missense(df_pssm.copy(), 'variant_id')
frame_clinvar

merge_missense: 62,040 raw records
merge_missense: 62,040 records matched to predictions
merge_missense: 21,176 annotated as destabilizing
merge_missense: 8,720 annotated with interfaces at pDockQ > 0.23
merge_missense: 29,900 annotated with pockets at pocket_score > 800


,variant_id,total energy,Backbone Hbond,Sidechain Hbond,Van der Waals,Electrostatics,Solvation Polar,Solvation Hydrophobic,Van der Waals clashes,entropy sidechain,...,energy Ionisation,am_pathogenicity,am_class,am_label,pred_ddg,pred_ddg_label,interface_pdockq,interface_label,pocket_label,mechanistic_label
0,P00533/K29A,0.259316,1.039020,0.863101,0.645481,-0.015303,-1.184350,0.788022,-0.007773,-1.375760,...,0.0,0.2953,benign,False,0.754909,False,NaN,False,True,Pockets
1,P00533/K29C,0.619045,0.961476,0.863101,0.223816,0.154480,-0.633215,0.367346,-0.007773,-0.968393,...,0.0,0.8289,pathogenic,True,1.150450,False,NaN,False,True,Pockets
2,P00533/K29D,0.402834,0.366827,0.341720,0.443973,0.060787,-0.605891,0.627430,-0.007899,-0.852986,...,0.0,0.6167,pathogenic,True,1.255870,False,NaN,False,True,Pockets
3,P00533/K29E,0.363608,0.138926,0.109867,0.107757,-0.010313,-0.122570,0.186013,-0.006306,-0.056120,...,0.0,0.2175,benign,False,0.799203,False,NaN,False,True,Pockets
4,P00533/K29F,0.550734,0.845414,0.863101,-0.221429,-0.014297,0.111640,-0.241121,0.004470,-0.666604,...,0.0,0.7423,pathogenic,True,0.498027,False,NaN,False,True,Pockets
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62035,Q9Y5Z9/I338S,0.905005,0.018868,0.000000,0.814935,0.000000,-0.735512,1.584240,-0.004999,-0.511685,...,0.0,0.2140,benign,False,1.040740,False,NaN,False,False,Unassigned
62036,Q9Y5Z9/I338T,0.786325,0.000000,0.000000,0.586331,0.000000,-0.417606,1.158290,0.017524,-0.423065,...,0.0,0.2302,benign,False,0.816912,False,NaN,False,False,Unassigned
62037,Q9Y5Z9/I338V,0.620887,0.000000,0.000000,0.330322,0.000000,-0.114172,0.735855,0.015844,-0.290465,...,0.0,0.0887,benign,False,0.680518,False,NaN,False,False,Unassigned
62038,Q9Y5Z9/I338W,-1.683120,0.018868,0.000000,-0.680052,-0.011924,-0.173973,-1.221620,0.025124,0.571906,...,0.0,0.3098,benign,False,-1.771450,False,NaN,False,False,Unassigned


In [4]:
# Compare to all foldx output columns; `total energy` closely matches `pred_ddg`
frame_clinvar[df_pssm.columns[1:]].corrwith(frame_clinvar['pred_ddg']).sort_values(ascending=False)

total energy             0.953999
Van der Waals clashes    0.868602
torsional clash          0.374735
Solvation Polar          0.355741
disulfide                0.243944
entropy mainchain        0.220985
energy Ionisation        0.138465
entropy sidechain        0.108507
backbone clash           0.051832
helix dipole             0.022109
cis_bond                -0.001460
Backbone Hbond          -0.002825
Electrostatics          -0.043627
Sidechain Hbond         -0.051786
Solvation Hydrophobic   -0.119891
Van der Waals           -0.215133
dtype: float64